In [0]:
from pyspark.sql.functions import round, lit, avg

# Declaring variables for mounting Azure Blob
container_name = "container_name"
storage_account_name = "storage_account_name"
storage_account_access_key = "storage_account_key"
url = "wasbs://" + container_name + "@" + storage_account_name + ".blob.core.windows.net"
config = "fs.azure.account.key."+ storage_account_name + ".blob.core.windows.net"

In [0]:
# Mounting azure storage
dbutils.fs.mount(
    source = url,
    mount_point = "/mnt/speedAnalysis",
    extra_configs = {config:storage_account_access_key}
)

Out[2]: True

In [0]:
# Defining schema for DF
from pyspark.sql.types import *
schema = StructType([ StructField("operator", StringType(), False),
                     StructField("speed_type", StringType(), False),
                     StructField("download_upload", StringType(), False),
                     StructField("speed_kbps", IntegerType(), False),
                     StructField("latency", IntegerType(), False),
                     StructField("state", StringType(), False)]
)

# Defining Schema for Final DF - Not Used
finalDF_schema = StructType([ StructField("operator", StringType(), False),
                     StructField("speed_type", StringType(), False),
                     StructField("download_upload", StringType(), False),
                     StructField("speed_kbps", IntegerType(), False),
                     StructField("latency", IntegerType(), False),
                     StructField("state", StringType(), False),
                     StructField("speed_mbps", StringType(), False),
                     StructField("month", StringType(), False),
                     StructField("year", StringType(), False)]
)
finalDF = sqlContext.createDataFrame(sc.emptyRDD(), finalDF_schema)

In [0]:
# Iterating through each file in the mounted directory

for file in dbutils.fs.ls("/mnt/speedAnalysis"):
    # Extracting file specific details
    file_path = file[0]
    month = [x for x in file[1].split()[0:2]][0]
    year = [x for x in file[1].split()[0:2]][1]
    print(f"Currently Processing file for {month} - {year}")
    
    # Reading csv file
    df = (spark.read.format("csv")
          .schema(schema)
          .load(file_path))
    
    # Removing null and NA values, calculation speed in MBPS
    cleaned_df = (df.filter(df.latency.isNotNull() & (df.state != 'NA') & (df.speed_kbps.isNotNull()))
                  .withColumn("speed_mbps", round(df.speed_kbps/1024,2))
                 )
    
    # Grouping all columns and aggregating numerical columns
    groupedDF = (cleaned_df.groupBy("operator", "speed_type", "download_upload","state")
             .agg(round(avg("speed_kbps"),2).alias("speed_kbps"), round( avg("latency"),2).alias("latency"), round(avg("speed_mbps"),2).alias("speed_mbps"))
             .orderBy("operator", "speed_type","state")
             .withColumn("month", lit(month))
             .withColumn("year", lit(year))
                )
    
    # Writing DF to csv file in append mode 
    groupedDF.write.format("csv").mode("append").save("/mnt/speedAnalysis/final/speed.csv")

Currently Processing file for April - 2018
Currently Processing file for April - 2019
Currently Processing file for April - 2020
Currently Processing file for Aug - 2019
Currently Processing file for August - 2018
Currently Processing file for August - 2020
Currently Processing file for Dec - 2019
Currently Processing file for Dec - 2020
Currently Processing file for December - 2018
Currently Processing file for Feb - 2019
Currently Processing file for Feb - 2020
Currently Processing file for Jan - 2020
Currently Processing file for January - 2019
Currently Processing file for July - 2018
Currently Processing file for July - 2019
Currently Processing file for July - 2020
Currently Processing file for June - 2018
Currently Processing file for June - 2019
Currently Processing file for June - 2020
Currently Processing file for March - 2018
Currently Processing file for March - 2019
Currently Processing file for March - 2020
Currently Processing file for May - 2018
Currently Processing file for May - 2019
Currently Processing file for May - 2020
Currently Processing file for Nov - 2019
Currently Processing file for Nov - 2020
Currently Processing file for November - 2018
Currently Processing file for Oct - 2019
Currently Processing file for Oct - 2020
Currently Processing file for October - 2018
Currently Processing file for Sept - 2019
Currently Processing file for Sept - 2020
Currently Processing file for September - 2018

--------------------------------------------------------------------------- 
 IndexError Traceback (most recent call last)
 <command-3439854025734438> in <module> 
 5 file_path = file [ 0 ] 
 6 month = [ x for x in file [ 1 ] . split ( ) [ 0 : 2 ] ] [ 0 ] 
 ----> 7 year = [ x for x in file [ 1 ] . split ( ) [ 0 : 2 ] ] [ 1 ] 
 8 print ( f"Currently Processing file for {month} - {year}" ) 
 9 

 IndexError : list index out of range

In [0]:
df2 = (spark.read.format("csv")
      .load("dbfs:/mnt/speedAnalysis/final/speed.csv"))
df2.show()

+------+---+--------+----------------+------+------+----+-----+----+
 _c0|_c1| _c2| _c3| _c4| _c5| _c6| _c7| _c8|
+------+---+--------+----------------+------+------+----+-----+----+
AIRTEL| 2G|download| Andhra Pradesh|208.26|-80.79| 0.2|April|2018|
AIRTEL| 2G| upload| Andhra Pradesh|190.39|-78.21|0.19|April|2018|
AIRTEL| 2G| upload| Assam|107.33| -81.0|0.11|April|2018|
AIRTEL| 2G|download| Assam| 277.0|-75.67|0.27|April|2018|
AIRTEL| 2G|download| Bihar|302.21|-62.43|0.29|April|2018|
AIRTEL| 2G| upload| Bihar|282.19| -66.0|0.28|April|2018|
AIRTEL| 2G| upload| Chennai| 264.5| -77.0|0.26|April|2018|
AIRTEL| 2G|download| Chennai|351.29|-76.43|0.34|April|2018|
AIRTEL| 2G|download| Delhi|348.33|-83.67|0.34|April|2018|
AIRTEL| 2G| upload| Delhi| 112.4| -76.2|0.11|April|2018|
AIRTEL| 2G|download| Gujarat|327.29|-78.71|0.32|April|2018|
AIRTEL| 2G| upload| Gujarat| 310.8| -69.4| 0.3|April|2018|
AIRTEL| 2G|download| Haryana| 315.5| -74.0|0.31|April|2018|
AIRTEL| 2G| upload| Haryana| 424.0| -74.0|0.42|April|2018|
AIRTEL| 2G| upload|Himachal Pradesh|234.25| -72.0|0.23|April|2018|
AIRTEL| 2G|download|Himachal Pradesh| 253.0| -69.0|0.25|April|2018|
AIRTEL| 2G| upload| Jammu & Kashmir| 250.5|-64.75|0.24|April|2018|
AIRTEL| 2G|download| Jammu & Kashmir|299.43|-60.43|0.29|April|2018|
AIRTEL| 2G|download| Karnataka| 419.8| -71.4|0.41|April|2018|
AIRTEL| 2G| upload| Karnataka|219.83|-80.33|0.21|April|2018|
+------+---+--------+----------------+------+------+----+-----+----+
only showing top 20 rows

In [0]:
df2.filter(df2._c7 == "Jan").show()

+------+---+--------+----------------+-------+------+-----+---+----+
 _c0|_c1| _c2| _c3| _c4| _c5| _c6|_c7| _c8|
+------+---+--------+----------------+-------+------+-----+---+----+
AIRTEL| 3G| upload| Andhra Pradesh| 774.58|-79.92| 0.76|Jan|2020|
AIRTEL| 3G|download| Andhra Pradesh|2275.98|-80.55| 2.22|Jan|2020|
AIRTEL| 3G|download| Assam| 367.0| -83.0| 0.36|Jan|2020|
AIRTEL| 3G| upload| Assam| 947.5| -83.0| 0.93|Jan|2020|
AIRTEL| 3G|download| Bihar|1564.13| -92.5| 1.53|Jan|2020|
AIRTEL| 3G| upload| Bihar| 496.13| -94.0| 0.48|Jan|2020|
AIRTEL| 3G|download| Chennai|5091.29|-73.86| 4.97|Jan|2020|
AIRTEL| 3G| upload| Chennai|1159.13|-78.25| 1.13|Jan|2020|
AIRTEL| 3G| upload| Delhi| 611.6|-81.06| 0.6|Jan|2020|
AIRTEL| 3G|download| Delhi|2492.83|-80.44| 2.43|Jan|2020|
AIRTEL| 3G| upload|Himachal Pradesh| 625.33|-89.89| 0.61|Jan|2020|
AIRTEL| 3G|download|Himachal Pradesh|3113.78|-91.67| 3.04|Jan|2020|
AIRTEL| 3G| upload| Karnataka| 774.0| -87.0| 0.76|Jan|2020|
AIRTEL| 3G|download| Karnataka|10371.0| -87.0|10.13|Jan|2020|
AIRTEL| 3G|download| Kerala| 3485.0|-102.0| 3.4|Jan|2020|
AIRTEL| 3G| upload| Kerala| 669.0|-103.0| 0.65|Jan|2020|
AIRTEL| 3G| upload| Maharashtra| 48.5| -63.0| 0.05|Jan|2020|
AIRTEL| 3G|download| Maharashtra| 2398.0| -57.0| 2.34|Jan|2020|
AIRTEL| 3G|download| North East| 1508.0| -85.0| 1.47|Jan|2020|
AIRTEL| 3G| upload| North East| 923.0| -87.0| 0.9|Jan|2020|
+------+---+--------+----------------+-------+------+-----+---+----+
only showing top 20 rows

In [0]:
df2.filter(df2._c7 == "December").count()

Out[11]: 342

In [0]:
dbutils.fs.unmount("/mnt/speedAnalysis")

In [0]:
df2.create 

Out[12]: 'wasbs://files@practiceblobs.blob.core.windows.net'

In [0]:
df2.write.format("csv").mode("overwrite").save("/FileStore/my-stuff/speed3.csv")

In [0]:
df2.coalesce(1).write.format("csv").save("/FileStore/my-stuff/speedFinal.csv")